<a href="https://colab.research.google.com/github/YoannRaguenes/Projet-Data-Science-Clubs-De-Foot-Anglais/blob/master/EnglishPlayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#English Player Dataset
The purpose of this code is to create a dataset of almost all professional football players in the English leagues.
We will catch the data from websites:

*   https://sofifa.com
*   https://fbref.com

we are going to use the library Beautifulsoup in order to collect informations on the website.

In [0]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd
from bs4.element import Tag 

###Find the stats of the players
We define our function, the goal of this function is to create a dataframe with some stats about the players.
The shape of the html code is the same for each page, there are tables with the stats for each player in the league.






In [0]:
def getPlayers(Html,div):
  html = urlopen(Html)
  html_soup = BeautifulSoup(html, 'html.parser')
  rows = html_soup.findAll("div",id="all_stats_standard")
  comments =rows[0].findAll(string=lambda text: isinstance(text, Comment))
  for c in comments:
    c.extract()

  sp = BeautifulSoup(c, 'html.parser')
  l = sp.table.tbody.findAll("tr")
  players = []
  for row in range(len(l)):
    if l[row].a!=None :
      player4 = {
                  "name": l[row].a.text,
                  "Nation": l[row].findAll("td",class_="left")[1].text,
                  "Pos": l[row].findAll("td",class_="center")[0].text,
                  "Squad": l[row].findAll("td",class_="left")[2].text,
                 "division": div,
                  "Age": l[row].findAll("td",class_="center")[1].text,
                  "Born": l[row].findAll("td",class_="center")[2].text,
               "MP" : l[row].findAll("td",class_="right group_start")[0].text,
               "Starts" : l[row].findAll("td",class_="right")[1].text,
               "Min" : l[row].findAll("td",class_="right")[2].text,
               "Gls" : l[row].findAll("td",class_="right group_start")[1].text,
               "Ast" : l[row].findAll("td",class_="right")[4].text,
               "PK" : l[row].findAll("td",class_="right")[5].text,
               "PKatt" : l[row].findAll("td",class_="right")[6].text,
               "CrdY" : l[row].findAll("td",class_="right")[7].text,
               "CrdR" : l[row].findAll("td",class_="right")[8].text,
               "Gls/90" : l[row].findAll("td",{"data-stat":"goals_per90"})[0].text,
               "Ast/90" : l[row].findAll("td",class_="right")[10].text,
               "G+A" : l[row].findAll("td",class_="right")[11].text,
               "G-PK" : l[row].findAll("td",class_="right")[12].text,
               "G+A-PK" : l[row].findAll("td",class_="right")[13].text,
               'Contract': None,
               "Value": None ,
               'Release_clause':None

              }
      players.append(player4)
  return players


We use the function for the different leagues

In [0]:
#england
df1 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/9/stats/Premier-League-Stats",1))
df2 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/10/stats/Championship-Stats",2))
df3 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/15/stats/League-One-Stats",3))
df4 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/16/stats/League-Two-Stats",4))
df5 = pd.DataFrame(getPlayers("https://fbref.com/en/comps/34/stats/National-League-Stats",5))
tot = pd.concat([df1,df2,df3,df4,df5])
tot[1:-1].to_csv("ENplayers.csv", encoding="utf-8-sig")


In [0]:
tot = pd.concat([df1,df2,df3,df4,df5])

Read the csv file 

In [0]:
tot[1:-1].to_csv("ENplayers.csv", encoding="utf-8-sig")

In [0]:
df = pd.read_csv('ENplayers.csv')
df

,Unnamed: 0,name,Nation,Pos,Squad,division,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls/90,Ast/90,G+A,G-PK,G+A-PK,Contract,Value,Release_clause
0,1,Max Aarons,eng ENG,DF,Norwich City,1,19.0,2000.0,27,27,"2,430",0.0,1.0,0,0,6,0,0.00,0.04,0.04,0.00,0.04,NaN,NaN,NaN
1,2,Tammy Abraham,eng ENG,FW,Chelsea,1,21.0,1997.0,25,23,"1,932",13.0,3.0,0,0,2,0,0.61,0.14,0.75,0.61,0.75,NaN,NaN,NaN
2,3,Che Adams,eng ENG,FW,Southampton,1,23.0,1996.0,22,8,731,0.0,2.0,0,0,0,0,0.00,0.25,0.25,0.00,0.25,NaN,NaN,NaN
3,4,Adrián,es ESP,GK,Liverpool,1,32.0,1987.0,11,9,875,0.0,0.0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
4,5,Sergio Agüero,ar ARG,FW,Manchester City,1,31.0,1988.0,22,17,"1,397",16.0,3.0,2,3,1,0,1.03,0.19,1.22,0.90,1.10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3202,670,Mark Yeates,ie IRL,"FW,MF",Fylde,5,34.0,1985.0,29,18,"1,682",1.0,NaN,0,0,3,0,0.05,NaN,NaN,0.05,NaN,NaN,NaN,NaN
3203,671,Luke Young,eng ENG,"MF,DF",Wrexham,5,26.0,1993.0,37,36,"3,244",3.0,NaN,0,0,5,0,0.08,NaN,NaN,0.08,NaN,NaN,NaN,NaN
3204,672,Adi Yussuf,tz TAN,"FW,MF",Boreham Wood,5,27.0,1992.0,3,0,26,0.0,NaN,0,0,0,0,0.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN
3205,673,Adi Yussuf,tz TAN,FW,Solihull Moors,5,27.0,1992.0,9,3,308,0.0,NaN,0,0,0,0,0.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN


Now we can do some requests and analysis with this dataframe.


In [0]:
df.loc[ (df['Age']<=20) & (df['Gls']>=10)]


,Unnamed: 0,name,Nation,Pos,Squad,division,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls/90,Ast/90,G+A,G-PK,G+A-PK,Contract,Value,Release_clause
880,389,Bryan Mbeumo,fr FRA,FW,Brentford,2,19.0,1999.0,34,29,"2,387",14.0,6.0,0,0,3,0,0.53,0.23,0.75,0.53,0.75,NaN,NaN,NaN
1375,206,Marcus Forss,fi FIN,FW,AFC Wimbledon,3,20.0,1999.0,18,17,"1,337",11.0,1.0,3,3,3,1,0.74,0.07,0.81,0.54,0.61,NaN,NaN,NaN


In [0]:
df.sort_values(by=["Gls"]).tail(10)

,Unnamed: 0,name,Nation,Pos,Squad,division,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls/90,Ast/90,G+A,G-PK,G+A-PK,Contract,Value,Release_clause
739,248,Lewis Grabban,eng ENG,FW,Nott'ham Forest,2,31.0,1988.0,36,34,"3,104",17.0,2.0,3,4,2,0,0.49,0.06,0.55,0.41,0.46,NaN,NaN,NaN
3041,509,John Rooney,eng ENG,"MF,DF",Barrow,5,28.0,1990.0,37,37,"3,207",17.0,NaN,4,4,6,1,0.48,NaN,NaN,0.36,NaN,NaN,NaN,NaN
3026,494,Jamie Reid,nir NIR,"FW,MF",Torquay,5,25.0,1994.0,32,32,"2,799",18.0,NaN,5,5,3,0,0.58,NaN,NaN,0.42,NaN,NaN,NaN,NaN
3132,600,Kabongo Tshimanga,eng ENG,"FW,MF",Boreham Wood,5,23.0,1996.0,37,37,"3,110",18.0,NaN,2,2,0,0,0.52,NaN,NaN,0.46,NaN,NaN,NaN,NaN
452,453,Jamie Vardy,eng ENG,FW,Leicester City,1,32.0,1987.0,26,25,"2,223",19.0,4.0,4,5,2,0,0.77,0.16,0.93,0.61,0.77,NaN,NaN,NaN
3011,479,Scott Quigley,eng ENG,"FW,DF",Barrow,5,26.0,1992.0,35,34,"2,806",20.0,NaN,0,0,3,1,0.64,NaN,NaN,0.64,NaN,NaN,NaN,NaN
1130,639,Ollie Watkins,eng ENG,FW,Brentford,2,23.0,1995.0,37,37,"3,318",22.0,2.0,0,1,3,0,0.60,0.05,0.65,0.60,0.65,NaN,NaN,NaN
903,412,Aleksandar Mitrović,rs SRB,FW,Fulham,2,24.0,1994.0,34,34,"3,049",23.0,1.0,2,3,8,0,0.68,0.03,0.71,0.62,0.65,NaN,NaN,NaN
1766,597,Ivan Toney,eng ENG,FW,P'borough Utd,3,23.0,1996.0,32,32,"2,839",24.0,5.0,2,2,10,0,0.76,0.16,0.92,0.70,0.86,NaN,NaN,NaN
2002,173,Eoin Doyle,ie IRL,FW,Swindon Town,4,31.0,1988.0,28,28,"2,492",25.0,4.0,2,2,6,0,0.90,0.14,1.05,0.83,0.98,NaN,NaN,NaN


In [0]:
df.loc[(df['Pos']== 'MF')&(df['Ast']>=10) & (df['Gls']>=5)].sort_values(by=["G+A"])

,Unnamed: 0,name,Nation,Pos,Squad,division,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls/90,Ast/90,G+A,G-PK,G+A-PK,Contract,Value,Release_clause
1092,601,John Swift,eng ENG,MF,Reading,2,24.0,1995.0,32,31,"2,643",5.0,10.0,0,0,6,1,0.17,0.34,0.51,0.17,0.51,NaN,NaN,NaN
2511,682,Randell Williams,eng ENG,MF,Exeter City,4,22.0,1996.0,37,33,"3,004",5.0,13.0,0,0,4,0,0.15,0.39,0.54,0.15,0.54,NaN,NaN,NaN
1378,209,Scott Fraser,sco SCO,MF,Burton Albion,3,24.0,1995.0,30,25,"2,292",5.0,11.0,0,0,2,0,0.20,0.43,0.63,0.20,0.63,NaN,NaN,NaN
100,101,Kevin De Bruyne,be BEL,MF,Manchester City,1,28.0,1991.0,26,25,"2,147",8.0,17.0,0,0,2,0,0.34,0.71,1.05,0.34,1.05,NaN,NaN,NaN
